In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
from collections import defaultdict
from pprint import pprint
from Bio import SeqIO
from Bio import Entrez

In [5]:
read_counts = pd.read_csv('../data/tidy_read_proportions.csv', index_col=0)

HIV_map = {
    'HIV_GAG': 790,
    'HIV_POL': 2085,
    'HIV_ENV': 6225,
    'HIV_REV': 5970,
    'HIV_NEF': 8797,
    'HIV_VPU': 6062,
    'HIV_VPR': 5559,
    'HIV_VIF': 5041, 
    'HIV_TAT': 5831,
}

def pivot_sites_sera(oligo_read_counts):
    
    oligos = oligo_read_counts.to_dict(orient="index").values() # { 'start': 1, 'end': 30, 'sera1':counts1, 'sera2':counts2,...}
    sites = []
    
    for oligo in oligos: 
        if 'HIV' in oligo['virus']:
            oligo['start'] += HIV_map[oligo['virus']]
            oligo['end'] += HIV_map[oligo['virus']]
            oligo['virus'] = 'HIV'
            
        start, end = oligo.pop('start'), oligo.pop('end')
        for site in range(start, end):
            site_record = oligo.copy()
            site_record['site'] = site
            sites.append(site_record) # { 'site': 0, 'sera1': counts1, 'sera2': counts2, ...}
        
    records = []
    for site in sites: 
        del site['Peptide_sequence']
        sera = [s for s in site.keys() if s not in ['site', 'virus', 'strains']]
        
        for s in sera:
            sera_site_record = { 
                'virus': site['virus'],
                'strains': site['strains'],
                'serum': s,
                'counts': site[s],
                'site': site['site']
                }
            records.append(sera_site_record)
            
    return pd.DataFrame(records, columns=['site', 'virus', 'strains', 'serum', 'counts'])


tidy_counts = pivot_sites_sera(read_counts)


In [ ]:
all_viruses = set(tidy_counts['virus'].values)
all_sera = set(tidy_counts['serum'].values)
print all_viruses
sns.set_style(style='whitegrid', rc={'palette':'pastel', 'font_scale':1.2})

fig, axes = plt.subplots(ncols=1, nrows=len(all_viruses), figsize=(15, 3*len(all_viruses)))
for virus, ax in zip(all_viruses, axes):
    for serum in all_sera:
        serum_counts = tidy_counts.loc[(tidy_counts['serum'] == serum) 
                                       & (tidy_counts['virus'] == virus)]
        sns.barplot(x='site', y='counts', data=serum_counts, saturation=0.3)
    ax.set_title(virus)
    ax.set_xlabel('Genomic Position')
    ax.set_ylabel('Proportion of Reads')

    

In [ ]:
''' WIP: eventually add pretty genome maps to the bottom of each virus read plot'''

# ### Parse Reference Accession Numbers from Docs

# reference_df = pd.read_csv('../docs/library_contents_tidy.csv', header=0)
# reference_df.dropna(how='any', inplace=True)

# reference_accessions = defaultdict(list)
# for idx, row in reference_df.iterrows():
#     reference_accessions[row['virus']]+=(row['accession'].split(', '))
# viruses = reference_accessions.keys()

# ### Download reference files from accessions
# def fetch_reference_records(accessions):
#     handle = Entrez.esearch(db="nuccore", term=" ".join(accessions))    # retrieve xml of search results
#     giList = Entrez.read(handle)['IdList'] # pull GI numbers from handle

#     # post NCBI query
#     try:
#         search_handle = Entrez.epost(db="nuccore", id=",".join(giList))
#         search_results = Entrez.read(search_handle)
#         webenv, query_key = search_results["WebEnv"], search_results["QueryKey"]
#         gb_files = Entrez.efetch(db="nuccore", rettype="gb", webenv=webenv, query_key=query_key)
#         seqs = [i for i in SeqIO.parse(gb_files, 'genbank')]
        
#     except RuntimeError:
#         print("ERROR: Couldn't connect with entrez, please run again")
#         print("Accessions:", accessions)
#         print("GIs:", giList)
#         return []
#     return seqs

# references = { virus: fetch_reference_records( reference_accessions[virus]) for virus in viruses}
# print {k: len(v) for k,v in references.items()}

In [3]:
# def parse_ref_map(reference):
#     ''' Parse gene locations from genbank reference files '''
#     gene_map = {}

#     for feature in reference.features:
#         try:
#             gene = feature.qualifiers['product'][0]
#             if gene != 'polyprotein':
#                 gene_map[gene] = (int(feature.location.start), int(feature.location.end))
#         except:
#             continue
#     return gene_map
    
# reference_maps = {virus: parse_ref_map(refs[0]) for virus, refs in references.items() if len(refs)}